In [140]:
import pandas as pd

In [141]:
df = pd.read_csv(r"C:\Users\capor\Downloads\dataset(in).csv")

In [142]:
df.drop(['Unnamed: 0','education.num', 'capital.gain', 'capital.loss'], axis=1, inplace=True)
import numpy as np

df.replace("?", np.nan, inplace=True)

In [143]:
df = df.dropna()

In [144]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'marital.status',
       'occupation', 'relationship', 'race', 'sex', 'hours.per.week',
       'native.country', 'income'],
      dtype='object')

In [148]:
df

,age,workclass,fnlwgt,education,marital.status,occupation,relationship,race,sex,hours.per.week,native.country,income
0,34,2,209297,11,4,6,1,2,1,40,38,0
1,32,2,107417,11,4,7,1,4,1,37,38,0
2,59,1,296253,12,0,9,4,4,0,60,38,1
3,44,0,102238,11,0,2,4,4,1,40,38,0
5,43,1,163434,9,2,10,0,4,1,55,38,1
...,...,...,...,...,...,...,...,...,...,...,...,...
29298,63,2,294009,9,2,9,0,4,1,45,38,1
29299,28,2,319768,12,2,11,0,4,1,45,9,1
29300,21,2,306114,15,4,0,3,4,1,20,38,0
29303,41,2,39581,15,0,0,1,2,0,24,7,0


In [145]:
from sklearn.preprocessing import LabelEncoder

# Lista de columnas a codificar
columnas_categoricas = [
    "workclass", "marital.status", "occupation", "relationship",
    "race", "sex", "native.country", "income", "education",
]

for col in columnas_categoricas:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])  # Transforma la columna# Guarda el encoder por si se necesita revertir después

# Ver los primeros registros transformados
print(df.head())

   age  workclass  fnlwgt  education  marital.status  occupation  \
0   34          2  209297         11               4           6   
1   32          2  107417         11               4           7   
2   59          1  296253         12               0           9   
3   44          0  102238         11               0           2   
5   43          1  163434          9               2          10   

   relationship  race  sex  hours.per.week  native.country  income  
0             1     2    1              40              38       0  
1             1     4    1              37              38       0  
2             4     4    0              60              38       1  
3             4     4    1              40              38       0  
5             0     4    1              55              38       1  


In [146]:
df

,age,workclass,fnlwgt,education,marital.status,occupation,relationship,race,sex,hours.per.week,native.country,income
0,34,2,209297,11,4,6,1,2,1,40,38,0
1,32,2,107417,11,4,7,1,4,1,37,38,0
2,59,1,296253,12,0,9,4,4,0,60,38,1
3,44,0,102238,11,0,2,4,4,1,40,38,0
5,43,1,163434,9,2,10,0,4,1,55,38,1
...,...,...,...,...,...,...,...,...,...,...,...,...
29298,63,2,294009,9,2,9,0,4,1,45,38,1
29299,28,2,319768,12,2,11,0,4,1,45,9,1
29300,21,2,306114,15,4,0,3,4,1,20,38,0
29303,41,2,39581,15,0,0,1,2,0,24,7,0


## Modelo

In [134]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import pickle  # Usaremos pickle para guardar y cargar los objetos

def train_and_evaluate_models(df, save_path="modelo_y_escalador.pkl"):
    # Definir X e y
    X = df.drop(columns=["income"])
    y = df["income"]

    # Dividir el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Escalar los datos
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Definir modelos e hiperparámetros
    models = {
        'Random Forest': RandomForestClassifier(random_state=0)
    }

    param_grid = {
        'Random Forest': {
            'n_estimators': [65, 70, 75, 100],
            'max_depth': [6, 7],
            'min_samples_split': [2, 3],
            'min_samples_leaf': [1, 2, 3],
        }
    }

    # Resultados
    results = []
    
    # Realizar Grid Search para cada modelo
    for name, model in models.items():
        print(f"Training {name}...")
        grid_search = GridSearchCV(model, param_grid[name], cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train_scaled, y_train)

        # Obtener mejor modelo y parámetros
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_

        # Evaluar en conjunto de prueba
        y_pred = best_model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        # Registrar resultados
        results.append({
            'Model': name,
            'Best Parameters': best_params,
            'Train Accuracy (CV)': best_score,
            'Test Accuracy': accuracy,
            'Classification Report': report
        })

        print(f"{name} - Best Parameters: {best_params}")
        print(f"{name} - Train Accuracy (CV): {best_score}")
        print(f"{name} - Test Accuracy: {accuracy}\n")
        print(report)

        # Guardar el mejor modelo y el escalador en un solo archivo .pkl
        with open(save_path, "wb") as f:
            pickle.dump({"modelo": best_model, "escalador": scaler}, f)

        print(f"Modelo y escalador guardados en '{save_path}'")

    # Convertir resultados a DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Llamar a la función y guardar el modelo y escalador en un solo archivo
results_df = train_and_evaluate_models(df, save_path="modelo_y_escalador.pkl")

Training Random Forest...
Random Forest - Best Parameters: {'max_depth': 7, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 100}
Random Forest - Train Accuracy (CV): 0.8240516179050668
Random Forest - Test Accuracy: 0.8252945508100147

              precision    recall  f1-score   support

           0       0.87      0.91      0.89      4120
           1       0.67      0.55      0.60      1312

    accuracy                           0.83      5432
   macro avg       0.77      0.73      0.75      5432
weighted avg       0.82      0.83      0.82      5432

Modelo y escalador guardados en 'modelo_y_escalador.pkl'


In [135]:
import pickle

# Cargar el modelo y el escalador desde el archivo .pkl
with open("modelo_y_escalador.pkl", "rb") as f:
    data = pickle.load(f)

modelo_cargado = data["modelo"]
escalador_cargado = data["escalador"]

print("Modelo y escalador cargados correctamente.")

Modelo y escalador cargados correctamente.


In [136]:
modelo_cargado

RandomForestClassifier(max_depth=7, min_samples_leaf=3, random_state=0)

In [137]:
escalador_cargado

StandardScaler()

In [157]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

# Cargar datos
df = pd.read_csv(r"C:\Users\capor\Downloads\dataset(in).csv")
df.drop(['Unnamed: 0', 'education.num', 'capital.gain', 'capital.loss'], axis=1, inplace=True)

# Limpieza de datos
df.replace("?", np.nan, inplace=True)
df = df.dropna()

# Codificar variables categóricas
columnas_categoricas = [
    "workclass", "marital.status", "occupation", "relationship",
    "race", "sex", "native.country", "income", "education"
]

for col in columnas_categoricas:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Definir X e y
X = df.drop(columns=["income"])
y = df["income"]

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar modelo XGBoost
modelo = XGBClassifier(random_state=42, n_estimators=100, max_depth=5, learning_rate=0.05)
modelo.fit(X_train, y_train)

# Evaluar modelo
y_pred = modelo.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

# Guardar modelo y LabelEncoders
with open("modelo_xgboost.pkl", "wb") as f:
    pickle.dump(modelo, f)

print("Modelo guardado correctamente.")

Precisión del modelo: 0.84
              precision    recall  f1-score   support

           0       0.87      0.92      0.90      6132
           1       0.71      0.60      0.65      2016

    accuracy                           0.84      8148
   macro avg       0.79      0.76      0.77      8148
weighted avg       0.83      0.84      0.84      8148

Modelo guardado correctamente.
